In [1]:
# Hàm để đặt một lệnh mua
import MetaTrader5 as mt5

def orderSendForex(data):
    # Lấy dữ liệu từ từ điển
    datetime = data['Datetime']
    open_price = data['Open']
    close_price = data['Close']
    volume = data['Volume']
    # sma = data['SMA']
    # short_ema = data['short_ema']
    # long_ema = data['long_ema']
    macd = data['MACD']
    # signal_line = data['Signal_Line']
    buy_signal = data['Buy_Signal']    
    sell_signal = data['Sell_Signal']
    symbol = data['Symbol'] # + 'c'
    insertdate = data['Insertdate']
    
    if not mt5.initialize(login=48333595, password='9$R#9ht@', server='HFMarketsGlobal-Demo'): # Hotforex Real
    # if not mt5.initialize(login=10138239, password='xiVUPi4J', server='mt5-b2broker-metatrader5.fastmt-b.net:443'): # B2Brocker Demo
    # if not mt5.initialize(login=51524754, password='I2dmjFp5', server='mt5-demo.icmarkets.com'): # ICMarket Demo
    # if not mt5.initialize(login=7174192, password='NttZS95a', server='ICMarketsSC-MT5-2.icmarkets.com'): # ICMarket Real
    # if not mt5.initialize(login=115407068, password='123456Qaz1!', server='mt5trial6.exness.com:443'): # Exness Trial    
        print("Initialize() failed, error code =", mt5.last_error())
        quit()
    else:
        if(buy_signal == 'True'):
            lot = 0.1  # Số lượng lô mà bạn muốn mua
            if not mt5.symbol_select(symbol, True): # 
                print(f"Failed to select {symbol}, error code =", mt5.last_error())
                return

            symbol_info = mt5.symbol_info(symbol)
            if symbol_info is None:
                print(f"{symbol} not found")
                return
            
            point = symbol_info.point
            price = mt5.symbol_info_tick(symbol).ask
            deviation = 20  # Độ lệch giá cho phép

            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": lot,
                "type": mt5.ORDER_TYPE_BUY,
                "price": price,
                "sl": price - 400 * point,  # Lệnh dừng lỗ (Stop Loss)
                "tp": price + 200 * point,  # Lệnh chốt lời (Take Profit)
                "deviation": deviation,
                "magic": 234000,
                "comment": "python script open",
                "type_time": mt5.ORDER_TIME_GTC,
                # "type_filling": mt5.ORDER_FILLING_IOC,
                "type_filling": mt5.ORDER_FILLING_FOK,
                
            }

            result = mt5.order_send(request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print("Failed to send order :", result.retcode, result._asdict())
            else:
                print("Order placed BUY successfully!", result)

            # Đóng kết nối với MT5
            mt5.shutdown()
        elif (sell_signal == 'True'):
            lot = 0.1  # Số lượng lô bạn muốn bán
            if not mt5.symbol_select(symbol, True):
                print(f"Failed to select {symbol}, error code =", mt5.last_error())
                return

            symbol_info = mt5.symbol_info(symbol)
            if symbol_info is None:
                print(f"{symbol} not found")
                return
            
            point = symbol_info.point
            price = mt5.symbol_info_tick(symbol).bid  # Sử dụng giá bid cho lệnh bán
            deviation = 20  # Độ lệch giá cho phép

            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": lot,
                "type": mt5.ORDER_TYPE_SELL,
                "price": price,
                "sl": price + 400 * point,  # Lệnh dừng lỗ (Stop Loss) cho lệnh bán
                "tp": price - 200 * point,  # Lệnh chốt lời (Take Profit) cho lệnh bán
                "deviation": deviation,
                "magic": 234000,
                "comment": "python script open",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
                # "type_filling": mt5.ORDER_FILLING_FOK,
                # ORDER_FILLING_RETURN 
            }

            result = mt5.order_send(request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                print("Failed to send order :", result.retcode, result._asdict())
            else:
                print("Order placed SELL successfully!", result)

            # Đóng kết nối với MT5
            mt5.shutdown()

In [2]:
import redis

def entryForex(hash):
    # Xử lý dữ liệu ở đây
    print("Dữ liệu đã được xử lý")

    # Tạo kết nối Redis
    r = redis.Redis(host='localhost', port=6379, db=0)

    # Hash key
    hash_key = hash

    # Đọc dữ liệu từ hash
    data = r.hgetall(hash_key)
    
    if data:
        # Chuyển đổi dữ liệu từ bytes sang chuỗi nếu cần
        data = {key.decode('utf-8'): value.decode('utf-8') for key, value in data.items()}
        # Xử lý dữ liệu
        orderSendForex(data)
        # Xóa hash sau khi xử lý
        r.delete(hash_key)
        print(f"Hash '{hash_key}' đã được xóa khỏi Redis.")
    else:
        print(f"Không tìm thấy dữ liệu cho hash '{hash_key}'.")

In [3]:
from datetime import datetime, timedelta
import schedule
import time

def schedule_entryForex():
    entryForex('OG_4 Chi Bao')

# Lên lịch hàm scan_market để chạy mỗi 15 phút
schedule.every(10).seconds.do(schedule_entryForex)

while True:
    schedule.run_pending()  # Hàm này được gọi liên tục để kiểm tra xem có công việc nào đã đến thời gian cần thực hiện hay không
    time.sleep(1)

Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_4 Chi Bao'.
Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_4 Chi Bao'.
Dữ liệu đã được xử lý
Order placed BUY successfully! OrderSendResult(retcode=10009, deal=33111288, order=51384667, volume=0.1, price=2046.48, bid=0.0, ask=0.0, comment='Request executed', request_id=109429091, retcode_external=0, request=TradeRequest(action=1, magic=234000, order=0, symbol='XAUUSDc', volume=0.1, price=2046.37, stoplimit=0.0, sl=2042.37, tp=2048.37, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))
Hash 'OG_4 Chi Bao' đã được xóa khỏi Redis.
Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_4 Chi Bao'.
Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_4 Chi Bao'.
Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_4 Chi Bao'.
Dữ liệu đã được xử lý
Không tìm thấy dữ liệu cho hash 'OG_4 Chi Bao'.
Dữ liệu đã được xử lý
Không tìm thấy dữ liệu 